## Milestone 2: Neural Network Baseline and Hyperparameter Optimization

LIS 640 - Introduction to Applied Deep Learning

Due 3/7/25

## **Overview**
In Milestone 1 you have:
1. **Defined a deep learning problem** where AI can make a meaningful impact.
2. **Identified three datasets** that fit your topic and justified their relevance.
3. **Explored and visualized** the datasets to understand their structure.
4. **Implemented a PyTorch Dataset class** to prepare data for deep learning.

In Milestone 2 we will take the next step and implement a neural network baseline based on what we have learned in class! For this milestone, please use one of the datasets you picked in the last milestone. If you pick a new one, make sure to do Steps 2 - 4 again. 


## **Step 1: Define Your Deep Learning Problem**


The first step is to be clear about what you want your model to predict. Is your goal a classification or a regression task? what are the input features and what are you prediction targets y? Make sure that you have a sensible choice of features and a sensible choice of prediction targets y in your dataloader.

**Write down one paragraph of justification for how you set up your DataLoader below. If it makes sense to change the DataLoader from Milestone 1, describe what you changed and why:**


My problem remains the same -- Distinguish the English text beding generated by Japanese Native speakers and English Native speakers by BERT embedding and MLP. 

However the original data was insufficient in quantity: there was 200-400 words for each text and 60 for English native speakers and 60 for Japanese speakers. Therfore, in order to make the data quantity engough for training, I took two steps: 1. I split the each data equally into 10 parts and store them into the dataframe, marking English natives ad 0, Japanese natives as 1. Then, I used GPT 2 fine tuning to generate new text. New generated datasets are 4000 sets in total-- 2000 English natives (labeled as 0) and 2000 Japanese natives(labeled as 1). Then, I combined the original dataset (splited) with the current generated dataset to ctreate a dataset with 5200 data points-- 2600 for English natives (labeled as 0) and 2000 for Japanese natives (labeled as 1).

#Dataframes being (original, generated, and combined are all in github: https://github.com/kuangzil/Data-for-Native-language-Identification/tree/Milestone2)

The fine tuning code is shown as follows:

In [59]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch

import pandas as pd
from sklearn.model_selection import train_test_split




# We already have the combined dataset with the content and labels 0=English, 1=Japanese
df = pd.read_csv(r"D:\Applied Deep Learning\data\combined_DF_new\combined.csv")

# add a prefix to the content based on the label (0=English, 1=Japanese)
df['conditioned_text'] = df.apply(
    lambda x: "[JPN] " + x['text'] if x['label'] == 1 else "[ENG] " + x['text'],
    axis=1#axis=1 means apply the function to each row
)

# split the dataset
train_df, _ = train_test_split(df, test_size=0.1)#test size is 10% of the dataset


# loading the pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token='<|endoftext|>')# pad token is used to pad the input sequences to the same length
model = GPT2LMHeadModel.from_pretrained('gpt2')# GPT2LMHeadModel is the GPT-2 model with a language modeling head

# using the conditioned text as input
train_texts = train_df['conditioned_text'].tolist()#converts the column to a list
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
# truncation=True: truncate the input sequences to the maximum length the model can accept
# padding=True: pad the input sequences to the same length
# max_length=512: maximum length of the input sequences


# self-defined dataset class to use the encodings as input
class ConditionalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
        
    def __len__(self):
        return self.encodings.input_ids.shape[0]
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings.input_ids[idx],
            
            'labels': self.encodings.input_ids[idx]  # LLM will still use the input_ids as labels
        }

train_dataset = ConditionalDataset(train_encodings)

# training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-conditional",
    num_train_epochs=3,#number of epochs to train the model
    per_device_train_batch_size=4,#batch size for training
    logging_steps=100,#number of steps to print the logs
    save_steps=500,
    learning_rate=2e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# fine-tuning
trainer.train()
model.save_pretrained("./gpt2-conditional")#save the fine-tuned model

def generate_samples(model,tokenizer, prefix, num_samples=2000, batch_size=10):#num_samples is the number of samples to generate, batch_size is the number of samples to generate in each batch
    generated = []
    model.eval()
    # move the model to GPU
    model=model.to('cuda')
    while len(generated) < num_samples:
        inputs = tokenizer.encode(prefix, return_tensors='pt').to('cuda')
        
        outputs = model.generate(
            inputs,
            max_length=100,#maximum length of the generated sequences
            num_return_sequences=batch_size,
            temperature=0.7,#temperature is used to control the randomness of the generated samples
            top_p=0.9,#top_p is used to control the diversity of the generated samples
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        
        for output in outputs:
            text = tokenizer.decode(output, skip_special_tokens=True)
            clean_text = text.replace(prefix, "").strip()
            if len(clean_text) > 10:  # filtering out the ones that are too short
                generated.append(clean_text)
                
            if len(generated) >= num_samples:
                break
                
    return generated[:num_samples]

# Generate Japanese samples
japanese_samples = generate_samples(model,tokenizer, "[JPN]", num_samples=2000)

# Generate English samples
english_samples = generate_samples(model,tokenizer, "[ENG]", num_samples=2000)

# setting up DataFrame and save
result_df = pd.DataFrame({
    'text': japanese_samples + english_samples,
    'label': [0]*2000 + [1]*2000
})

result_df.to_csv(r"D:\Applied Deep Learning\data\combined_DF_new\generated_samples.csv", index=False)


oringinal = pd.read_csv(r"D:\Applied Deep Learning\data\combined_DF_new\combined.csv")
combined_df= pd.concat([oringinal, result_df], ignore_index=True)
combined_df.to_csv(r"D:\Applied Deep Learning\data\combined_DF_new\combined_new_df.csv", index=False)

 13%|█▎        | 100/759 [00:31<03:26,  3.19it/s]

{'loss': 3.0713, 'grad_norm': 6.13089656829834, 'learning_rate': 1.7364953886693017e-05, 'epoch': 0.4}


 26%|██▋       | 200/759 [01:05<03:10,  2.93it/s]

{'loss': 2.7528, 'grad_norm': 6.864502906799316, 'learning_rate': 1.4729907773386036e-05, 'epoch': 0.79}


 40%|███▉      | 300/759 [01:40<02:42,  2.83it/s]

...

{'loss': 2.5333, 'grad_norm': 6.608233451843262, 'learning_rate': 4.1897233201581036e-06, 'epoch': 2.37}


 92%|█████████▏| 700/759 [04:19<00:33,  1.74it/s]

{'loss': 2.544, 'grad_norm': 6.947583198547363, 'learning_rate': 1.5546772068511201e-06, 'epoch': 2.77}


100%|██████████| 759/759 [04:56<00:00,  2.56it/s]


{'train_runtime': 296.2855, 'train_samples_per_second': 10.237, 'train_steps_per_second': 2.562, 'train_loss': 2.6610089931563428, 'epoch': 3.0}


As the data quantity is insufficient, I fine tuned GPT 2 for text generation-- first, I split 10%  in the original dataset as validation set and what is left (90%) of orginal data set as the training set.  Then, I tried to tell GPT2 which part of the input sequences should be attended to and which part of the input should be ignored by input_ids.  Then, I trained the model and used it for text generation-- original data set contains 1230 datasets, 615 Japanese Natives and 615 Enlgish Natives. I generated 4000, 2000 for each according to the trained data.  And I combined the generated dataset and the original dataset with their labels-- 0 for English Natives and 1 for Japanese. 
The following cell is how I combined the dataset.

In [60]:
oringinal = pd.read_csv(r"D:\Applied Deep Learning\data\combined_DF_new\combined.csv")
combined_df= pd.concat([oringinal, result_df], ignore_index=True)
combined_df.to_csv(r"D:\Applied Deep Learning\data\combined_DF_new\combined_new_df.csv", index=False)
print(combined_df.head())

                                                text  label
0  file00201.txt 2001-01-16 male 20 Japan 1m stud...      1
1  file00202.txt 2001-01-17 female 19 Japan 1m st...      1
2  file00203.txt 2001-02-02 male 27 Japan 3 neigh...      1
3  file00204.txt 2001-02-02 female 31 Japan 1_12m...      1
4  file00205.txt 2001-02-02 female Japan 5 restau...      1


## **Step 2: Train a Neural Network in PyTorch**

We learned in class how to implement and train a feed forward neural network in pytorch. You can find reference implementations [here](https://github.com/mariru/Intro2ADL/blob/main/Week5/Week5_Lab_Example.ipynb) and [here](https://www.kaggle.com/code/girlboss/mmlm2025-pytorch-lb-0-00000). Tip: Try to implement the neural network by yourself from scratch before looking at the reference.


In [61]:
# imports
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from transformers import BertTokenizer, DistilBertModel
import numpy as np
import os
import pandas as pd
from torch.utils.data import Dataset
import torch.optim as optim
import umap.umap_ as umap 
# define dataloaders: make sure to have a train, validation and a test loader
#loading the pre-trained model tokenizer of BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')



print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name())
# load the pre-trained BERT model to GPU

device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model= DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)
# do not feed the data directly to the variable called"model"! in the training process!

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataset = pd.read_csv(dataframe)
        self.texts = self.dataset[['text']].astype(str)
        # Convert labels to long
        self.labels = self.dataset['label']#map({"Native English": 0, "Native Japanese": 1}).astype(int) this is no longer needed because I already labeled the df

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idex):
        text = self.texts.iloc[idex].values[0]
        label = torch.tensor(self.labels.iloc[idex], dtype=torch.long).to(device)  # Make sure it's long
        tokenized_text = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors="pt")
        tokenized_text = {key: val.to(device) for key, val in tokenized_text.items() if key != 'token_type_ids'}
        
        with torch.no_grad():
            output = model(**tokenized_text)
        
        embedding = output.last_hidden_state[:, 0, :].squeeze(0)
        embedding = embedding.float()  # Ensure embedding is float
        
        return embedding, label
    

# Example usage
dataset = CustomDataset(r"D:\Applied Deep Learning\data\combined_DF_new\combined_new_df.csv")
embedding, label = dataset[0]
print(embedding, label)
# Create a DataLoader
train_loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False)
#batch size too big, will cause memory error on GPU.
#we need to reduce the batch size to 4 or 8



True
1
NVIDIA GeForce RTX 4070 Laptop GPU
tensor([-1.8382e-01, -1.9284e-02,  2.4736e-01, -1.4338e-01, -1.3177e-01,
        -5.4348e-02,  1.5631e-01,  3.3115e-01, -2.8438e-01, -3.9994e-01,
         4.8445e-02, -6.1485e-02,  1.1324e-01,  1.7800e-01, -5.6604e-02,
         1.5154e-01,  2.2107e-01,  3.2030e-01,  9.4237e-02,  1.3604e-01,
         1.5238e-01, -2.5460e-01,  4.4725e-01,  1.7563e-02, -5.0431e-02,
        -8.8167e-02, -7.7577e-02,  1.8742e-01,  3.1459e-02, -8.7420e-02,
        -1.3092e-01,  2.4539e-01, -4.1070e-02, -1.9942e-01,  7.3461e-02,
        -2.7940e-01,  6.7753e-02, -1.4762e-01,  1.1093e-01,  1.4603e-01,
        -8.7253e-02, -3.2385e-02, -2.7490e-01, -3.2436e-01, -1.2498e-02,
        -2.6295e-01, -3.2339e+00, -2.8927e-02,  1.5797e-02, -2.5329e-01,
         4.7867e-01, -1.0998e-01, -9.2680e-02,  2.0057e-01,  2.7781e-01,
         3.7869e-01, -2.3559e-01,  3.4312e-01, -1.5666e-01, -9.3389e-02,
         6.3809e-01, -7.8705e-02, -4.3797e-01,  9.1733e-02, -3.3742e-03,
        -

In this cell, I used BERT-base-uncased and distill-bert-uncased for tokenization and embedding transfer. the transfered embedding, with their labels have been store in a dataframe. Then, I created a Dataset class to store the dataset, tokenization and embedding vectorization. Then, I split the dataset into train_loader and test_loader. 

In [84]:
# UMAP demonstration reduction to 3 demensions
import matplotlib.pyplot as plt
import umap.umap_ as umap  # make sure I am using umap-learn
import numpy as np
# Extract embeddings and labels
def get_embeddings_and_labels(loader):
    embeddings = []
    labels = []
    for batch_embeddings, batch_labels in loader:
        embeddings.append(batch_embeddings.cpu().numpy())  # transfer the embeddings to the CPU and to numpy
        labels.append(batch_labels.cpu().numpy())
    embeddings = np.vstack(embeddings)
    labels = np.hstack(labels)
    return embeddings, labels

# Extract embeddings and labels from the train and test loaders
train_embeddings, train_labels = get_embeddings_and_labels(train_loader)
test_embeddings, test_labels = get_embeddings_and_labels(test_loader)

# reduce the embeddings to 20 dimensions
umap_model = umap.UMAP(n_components=30, random_state=42)

train_embeddings_3d = umap_model.fit_transform(train_embeddings)
test_embeddings_3d = umap_model.transform(test_embeddings)  # use the same model to transform the test embeddings


c:\Users\likua\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\likua\AppData\Local\Programs\Python\Python312\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\Users\likua\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


## Demension Reduction!
As the 768 demension would highly increase the time for computation, I reduced the demension of embeddings into 30 for testing-- so that there would be a bit more information stored and highly increase the speed of computation. Then, I still stored the new training set and testing set into new dataloaders. I firstly transferred them into 3d, but I soon found out that there are too little information being stored and not meaningful for training with that dataset.
the class of 30 demension is in the following cell

In [85]:
# defining a new class of data set that stores the embeddings and labels 
from torch.utils.data import DataLoader
class EmbeddingDataset(torch.utils.data.Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.embeddings[idx], dtype=torch.float32), torch.tensor(self.labels[idx])

# setting up the data loaders for training and testing
train_embedding_dataset = EmbeddingDataset(train_embeddings_3d, train_labels)
test_embedding_dataset = EmbeddingDataset(test_embeddings_3d, test_labels)

train_loader_20d = DataLoader(train_embedding_dataset, batch_size=32, shuffle=True)
test_loader_20d = DataLoader(test_embedding_dataset, batch_size=32, shuffle=False)

In [94]:

# define the model
class MyNeuralNetwork(nn.Module):
    def __init__(self,hidden_size=185
                 ,dropout_rate=0.3267140666269757
                 ,learning_rate=0.00025937129423859296
                 ):
        super(MyNeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(768
                      , hidden_size),#input layer has 768 neurons which is the size of the BERT embedding (if dealing with 30 d embeddings, the input layer has 30 neurons)
            nn.BatchNorm1d(hidden_size),# adding batch normalization
            nn.Dropout(dropout_rate),# setting up dropout rate to 0.1
            nn.ReLU(),
            nn.Linear(hidden_size, 2)
        )
        self.learning_rate=learning_rate

# I used linear activation functions for the hidden layers and softmax for the output layer
# the input layer has 768 neurons which is the size of the BERT embedding
# the hidden layer has 100 neurons
# the output layer has 2 neurons
# I used the ReLU activation function for the hidden layer and the softmax activation function for the output because it is a binary classification problem and non-linear
# activation functions are needed for the hidden layers
        
    def forward(self, x):
        x = self.flatten(x)  # Flatten input if necessary
        x = self.linear_relu_stack(x)
        return x  # Output logits (CrossEntropyLoss will apply softmax internally)
    
    def fit(self, train_loader,val_loader, epochs,learning_rate=0.01,patience=5):
        self.to(device)
        self.train()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)# here I used the Adam optimizer because it is better for NLP
        best_val_loss= float("inf")#initialize the best validation loss to infinity
        patience_counter=0#initialize the patience counter to 0
        for epoch in range(epochs):
            for i, (data, labels) in enumerate(train_loader):
                data, labels = data.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = self(data)
                loss = criterion(outputs, labels)#calculate the loss
                loss.backward()#backpropagation
                optimizer.step()
                if i % 100 == 0:
                    print(f"Epoch: {epoch}, Loss: {loss.item()}")
            val_loss=self.evaluate(val_loader, criterion)#evaluate the model on the validation set
            if val_loss<best_val_loss:
                best_val_loss=val_loss#update the best validation loss

                patience_counter=0
            else:
                patience_counter+=1
            if patience_counter>patience:
                print("Early stopping")
                break
        return best_val_loss#return the best validation loss
    #in this funciton, we do not need to convert the data into tensors again bc we arleady did that
    def predict(self, x):
      with torch.no_grad():
        x = torch.tensor(x).to(device)
        outputs = self(x)
        y_pred = torch.argmax(outputs, dim=1)
        return np.where(y_pred.detach().cpu().numpy()==1, "Native Japanese", "Native English")
      
      
    def evaluate(self, val_loader,criterion):
        self.eval()
        total_loss = 0
        with torch.no_grad():
            for data, labels in val_loader:
                data, labels = data.to(device), labels.to(device)
                outputs = self(data)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
        return total_loss / len(val_loader)
        
    


In [95]:
#training the model 
def train_loop(dataloader, model, loss_fn, optimizer, device):
    """
    Runs one full training epoch on the given model using the provided dataloader.

    Parameters:
        dataloader (torch.utils.data.DataLoader): DataLoader providing batches of training data (inputs and labels).
        model (torch.nn.Module): The PyTorch model to be trained.
        loss_fn (function): The loss function used to compute the error between predictions and true labels.
        optimizer (torch.optim.Optimizer): The optimizer used to update the model parameters.
        device (torch.device): The device to run computations on (e.g., "cuda" or "cpu").

    Returns:
        None
    """
    size = len(dataloader.dataset)
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch, (X, y) in enumerate(dataloader):
        # Move data to the specified device
        X, y = X.to(device), y.to(device,dtype=torch.long)
        y=y.long()
        X=X.float()
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()  # Clear gradients
        loss.backward()        # Compute gradients
        optimizer.step()       # Update weights

        total_loss += loss.item()

        if batch % 100 == 0:
            current = batch * len(X)
            print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]")

    avg_loss = total_loss / len(dataloader)
    print(f"Average Training Loss: {avg_loss:.4f}")


def test_loop(dataloader, model, loss_fn, device):
    """
    Evaluates the model's performance on a test dataset.

    Parameters:
        dataloader (torch.utils.data.DataLoader): DataLoader providing batches of test data (inputs and labels).
        model (torch.nn.Module): The PyTorch model to be evaluated.
        loss_fn (function): The loss function used to compute the error between predictions and true labels.
        device (torch.device): The device to run computations on (e.g., "cuda" or "cpu").

    Returns:
        None
    """
    model.eval()  # Set the model to evaluation mode
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():  # Disable gradient calculations
        for X, y in dataloader:
            X, y = X.to(device), y.to(device,dtype=torch.long) # Move data to the specified device
            X=X.float()
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            

    avg_loss = test_loss / num_batches
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {(100 * accuracy):>0.1f}%, Avg loss: {avg_loss:>8f} \n")

batch_size = 4
# we set up batch size as 1 to avoid compurational catastrophy
# Instantiate custom model
my_model = MyNeuralNetwork().to(device)

# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(my_model.parameters(), lr=0.00019466370761380482)

# Training loop
epochs = 100
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_loader, my_model, loss_fn, optimizer, device)
    test_loop(test_loader, my_model, loss_fn, device)


Epoch 1
-------------------------------
loss: 0.964580  [    0/ 5124]
loss: 0.566610  [  400/ 5124]
loss: 0.894764  [  800/ 5124]
loss: 0.741076  [ 1200/ 5124]
loss: 0.510969  [ 1600/ 5124]
loss: 0.402552  [ 2000/ 5124]
loss: 0.684575  [ 2400/ 5124]
loss: 0.250913  [ 2800/ 5124]
loss: 0.698820  [ 3200/ 5124]
loss: 0.480867  [ 3600/ 5124]
loss: 0.509612  [ 4000/ 5124]
loss: 0.483319  [ 4400/ 5124]
loss: 0.742746  [ 4800/ 5124]
Average Training Loss: 0.5555
Test Error: 
 Accuracy: 79.9%, Avg loss: 0.456661 

Epoch 2
-------------------------------
loss: 0.509154  [    0/ 5124]
loss: 0.479380  [  400/ 5124]
loss: 0.626528  [  800/ 5124]
loss: 0.397814  [ 1200/ 5124]
loss: 0.283879  [ 1600/ 5124]
loss: 0.442565  [ 2000/ 5124]
loss: 0.603289  [ 2400/ 5124]
loss: 0.541206  [ 2800/ 5124]
loss: 0.175751  [ 3200/ 5124]
loss: 0.303028  [ 3600/ 5124]
loss: 0.488987  [ 4000/ 5124]
loss: 0.562255  [ 4400/ 5124]
loss: 0.382352  [ 4800/ 5124]
Average Training Loss: 0.5030
Test Error: 
 Accuracy: 80.6

#Initial result report

In this step, I built up my MLP and tried to training the model with the hidden_layer of 768 first (because I converted the data into BERT embeddings), but in order to deal with 30 demension embeddings, I converted the Linear function input as 30. 
first 30 d embedding training hidden_size=203,dropout_rate=0.1,learning_rate=0.0001
The result for the 100 epoch shows that :

Epoch 1
-------------------------------
loss: 0.584721  [    0/ 5124]
loss: 0.519337  [ 3200/ 5124]
Average Training Loss: 0.5618
Test Error: 
 Accuracy: 74.0%, Avg loss: 0.511170 

Epoch 2
-------------------------------
loss: 0.497146  [    0/ 5124]
loss: 0.446626  [ 3200/ 5124]
Average Training Loss: 0.5025
Test Error: 
 Accuracy: 74.5%, Avg loss: 0.481287 

Epoch 3
-------------------------------
loss: 0.492960  [    0/ 5124]
loss: 0.443114  [ 3200/ 5124]
Average Training Loss: 0.4762
Test Error: 
 Accuracy: 75.3%, Avg loss: 0.466697 

Epoch 4
...
Average Training Loss: 0.4308
Test Error: 
 Accuracy: 78.5%, Avg loss: 0.425507 


Accuracy improved from 74.0% to 78.5% for 100 epochs, and loss changed from 0.51 - 0.43. 


As this is the result for 30d data, the result is different from 768 d embedding training outcome: Underthe same circumsatnce, the result for 768 bert embedding is: accuracy: 79% --> 86.4% loss: 0.472- 0.373 Which means that there are obviously more information contained in the 768 demension result, however it takes 240 minutes for training while 30d used 27 seconds. This is because less demensions will cost less in computational resources  but wil lost more information. 


## **Step 2 continued: Try Stuff**

Use your code above to try different architectures. Make sure to use early stopping! Try adding Dropout and BatchNorm, try different learning rates. How do they affect training and validation performance? 

 **Summarize your observations in a paragraph below:**


When I set up the drop out rate as 0.2, the training speed is a bit faster than 0.1 of Drop out rate. But the loss is likely to increase in some of the epoch. For Dropout rate of 0.1, and batch norm to 100, it takes 30 minutes to train 30 epoches. But with drop out rate of 0.2, the trainig times is 28mingutes. However, the improvement in accuracy is also different-- for the former one with 0.1 dropout rate, the accuracy is from 79% --> 86.4%, while for the later one with drop out rate of 0.2, the improvement is 80% --> 83.3%. The change in loss is also different-- in the dropout of 0.1, the loss change is from 0.51-0.44 while for the drop out rate of 0.2, the loss change is from 0.472- 0.373. It is significant that lower drop out rate would make loss drop higher bacause there are less information lost.  I set up early stopping with the patience of 5, but the training process still proceed to the last.

Therefore, conclusion should be drawn that lower drop out rate (0.1 for example) could be the optimized solution to my MLP training because less information is likely to be droped and thus more information would be learned. On the contray, higher dropout rate (0.2) would result more information to be lost and inhibit the learning process. 

Then, I tried a different learning rate-- for the lr of 0.01, the learning speed is faster than the lr of 0.001. But under sthe same circumstance, higher learning rate would result less accurate result: under the same circunstance(10 epoch, dropout=10, same batch norm), lr of 0.01 would result 86.4% accuracy, while lr of 0.001 would result in 87.5% accuarcy.  


## **Step 3: Hyperparameter Optimization with Optuna**

As you can see, hyperparameter optimization can be tedious. In class we used [optuna](https://optuna.org/#code_examples) to automate the process. Your next task is to wrap your code from Step 2 into an objective which you can then optimize with optuna. Under the [code exaples](https://optuna.org/#code_examples) there is a tab *PyTorch* which should be helpful as it provides a minimal example on how to wrap PyTorch code inside an objective.

**Important: Make sure the model is evaluated on a validation set, not the training data!!**


In [ ]:
import optuna

def objective(trial):
    # Setting up the hyperparameters in the objective function
    hidden_size = trial.suggest_int("hidden_size", 50, 300)  # Hidden size between 50 and 300
    dropout_rate = trial.suggest_float("dropout", 0.01, 0.4)  # Dropout between 0.01 and 0.4
    learning_rate = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Learning rate between 1e-5 and 1e-2

    # Setting up the model with the suggested hyperparameters
    model = MyNeuralNetwork(hidden_size, dropout_rate, train_loader_20d).to(device)

    # Training the model (pass the correct data loaders)
    model.fit(train_loader_20d, val_loader=test_loader_20d, epochs=100, learning_rate=learning_rate)

    # Evaluate the model on the test_loader (we use test_loader to evaluate performance)
    val_loss = model.evaluate(test_loader_20d, criterion = nn.CrossEntropyLoss())  # Make sure I am using test_loader in the evaluation
    # what is more, I cannot connect "crtierion" with the previous one, so I just connected it with nn.CrossEntropyLoss()

    return val_loss  # Minimize the validation loss

# Create and run the Optuna optimization study
study = optuna.create_study(direction="minimize")  # Minimize the validation loss
study.optimize(objective, n_trials=50)  # Number of trials for optimization

# Print the best hyperparameters found by Optuna
print(study.best_params)





[I 2025-03-03 15:16:46,019] A new study created in memory with name: no-name-96810c6a-9540-4917-8093-c41bb9691905
C:\Users\likua\AppData\Local\Temp\ipykernel_34484\3251040904.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Learning rate between 1e-5 and 1e-2


Epoch: 0, Loss: 0.6610797643661499
Epoch: 0, Loss: 0.359302818775177
Epoch: 1, Loss: 0.3919389247894287
Epoch: 1, Loss: 0.4813230335712433
Epoch: 2, Loss: 0.3904144763946533
Epoch: 2, Loss: 0.23281002044677734
Epoch: 3, Loss: 0.3842809200286865
Epoch: 3, Loss: 0.4154621958732605
Epoch: 4, Loss: 0.36802324652671814
Epoch: 4, Loss: 0.30847716331481934
Epoch: 5, Loss: 0.589328944683075
Epoch: 5, Loss: 0.27581489086151123
Epoch: 6, Loss: 0.5492057800292969
Epoch: 6, Loss: 0.4563789367675781
Epoch: 7, Loss: 0.4560481905937195
Epoch: 7, Loss: 0.4762313663959503
Epoch: 8, Loss: 0.36396971344947815
Epoch: 8, Loss: 0.37339210510253906
Epoch: 9, Loss: 0.40023931860923767
Epoch: 9, Loss: 0.5566477179527283
Epoch: 10, Loss: 0.465040385723114
Epoch: 10, Loss: 0.3177785277366638
Epoch: 11, Loss: 0.38588738441467285
Epoch: 11, Loss: 0.3154771625995636
Epoch: 12, Loss: 0.43869513273239136
Epoch: 12, Loss: 0.4519065022468567
Epoch: 13, Loss: 0.46726712584495544
Epoch: 13, Loss: 0.5844473838806152
Epoch

[I 2025-03-03 15:17:03,108] Trial 0 finished with value: 0.41486448658809577 and parameters: {'hidden_size': 276, 'dropout': 0.32243513364407383, 'lr': 0.00010612609591523706}. Best is trial 0 with value: 0.41486448658809577.


Early stopping
Epoch: 0, Loss: 0.7873765230178833
Epoch: 0, Loss: 0.3893071711063385
Epoch: 1, Loss: 0.4720616340637207
Epoch: 1, Loss: 0.4370037615299225
Epoch: 2, Loss: 0.4287753105163574
Epoch: 2, Loss: 0.33902209997177124
Epoch: 3, Loss: 0.25430965423583984
Epoch: 3, Loss: 0.3418639004230499
Epoch: 4, Loss: 0.5559511184692383
Epoch: 4, Loss: 0.4378761649131775
Epoch: 5, Loss: 0.45901980996131897
Epoch: 5, Loss: 0.4202910363674164
Epoch: 6, Loss: 0.4501282274723053
Epoch: 6, Loss: 0.3754114806652069
Epoch: 7, Loss: 0.46161001920700073
Epoch: 7, Loss: 0.4323728084564209
Epoch: 8, Loss: 0.46950238943099976
Epoch: 8, Loss: 0.3815102279186249
Epoch: 9, Loss: 0.5094742178916931
Epoch: 9, Loss: 0.37362152338027954
Epoch: 10, Loss: 0.4727054834365845
Epoch: 10, Loss: 0.5619612336158752
Early stopping


[I 2025-03-03 15:17:06,078] Trial 1 finished with value: 0.42835562489926815 and parameters: {'hidden_size': 79, 'dropout': 0.3451948693871122, 'lr': 0.008500581192678849}. Best is trial 0 with value: 0.41486448658809577.


...

Epoch: 0, Loss: 0.819193959236145
Epoch: 0, Loss: 0.37868186831474304
Epoch: 1, Loss: 0.2224918007850647
Epoch: 1, Loss: 0.5634194016456604
Epoch: 2, Loss: 0.448588490486145
Epoch: 2, Loss: 0.41938695311546326
Epoch: 3, Loss: 0.540790855884552
Epoch: 3, Loss: 0.3782906234264374
Epoch: 4, Loss: 0.45985284447669983
Epoch: 4, Loss: 0.4827806055545807
Epoch: 5, Loss: 0.4091682434082031
Epoch: 5, Loss: 0.43858450651168823
Epoch: 6, Loss: 0.38509616255760193
Epoch: 6, Loss: 0.5257836580276489
Epoch: 7, Loss: 0.33883848786354065
Epoch: 7, Loss: 0.3446692228317261
Epoch: 8, Loss: 0.36768192052841187
Epoch: 8, Loss: 0.3743918240070343
Epoch: 9, Loss: 0.5035917162895203
Epoch: 9, Loss: 0.36440473794937134
Epoch: 10, Loss: 0.3789946138858795
Epoch: 10, Loss: 0.3985324501991272
Epoch: 11, Loss: 0.42013466358184814
Epoch: 11, Loss: 0.4733745753765106
Epoch: 12, Loss: 0.413896381855011
Epoch: 12, Loss: 0.484529584646225
Epoch: 13, Loss: 0.5250592231750488
Epoch: 13, Loss: 0.5311992168426514
Epoch: 1

[I 2025-03-03 15:26:41,322] Trial 48 finished with value: 0.41597702891583216 and parameters: {'hidden_size': 262, 'dropout': 0.21974287493850617, 'lr': 0.0002444047659671073}. Best is trial 22 with value: 0.41142106168145626.


Early stopping
Epoch: 0, Loss: 0.7474607229232788
Epoch: 0, Loss: 0.4360671937465668
Epoch: 1, Loss: 0.3226762115955353
Epoch: 1, Loss: 0.3721334934234619
Epoch: 2, Loss: 0.3660534620285034
Epoch: 2, Loss: 0.4229695498943329
Epoch: 3, Loss: 0.5197665095329285
Epoch: 3, Loss: 0.3630763590335846
Epoch: 4, Loss: 0.2883955240249634
Epoch: 4, Loss: 0.4346677362918854
Epoch: 5, Loss: 0.49669063091278076
Epoch: 5, Loss: 0.2768736481666565
Epoch: 6, Loss: 0.3587530255317688
Epoch: 6, Loss: 0.5562158226966858
Epoch: 7, Loss: 0.41625258326530457
Epoch: 7, Loss: 0.44215330481529236
Epoch: 8, Loss: 0.38214632868766785
Epoch: 8, Loss: 0.4937567114830017
Epoch: 9, Loss: 0.4484466314315796
Epoch: 9, Loss: 0.3577127754688263
Epoch: 10, Loss: 0.4205235242843628
Epoch: 10, Loss: 0.4780934453010559
Epoch: 11, Loss: 0.31808555126190186
Epoch: 11, Loss: 0.40855133533477783
Epoch: 12, Loss: 0.40249010920524597
Epoch: 12, Loss: 0.42081716656684875
Epoch: 13, Loss: 0.27167925238609314
Epoch: 13, Loss: 0.58412

[I 2025-03-03 15:26:54,399] Trial 49 finished with value: 0.41497972464732685 and parameters: {'hidden_size': 276, 'dropout': 0.26739005008931604, 'lr': 0.00014392726926503157}. Best is trial 22 with value: 0.41142106168145626.


{'hidden_size': 185, 'dropout': 0.3267140666269757, 'lr': 0.00025937129423859296}


Optuna shows the result that:

[I 2025-03-03 15:26:54,399] Trial 49 finished with value: 0.41497972464732685 and parameters: {'hidden_size': 276, 'dropout': 0.26739005008931604, 'lr': 0.00014392726926503157}. Best is trial 22 with value: 0.41142106168145626.

{'hidden_size': 185, 'dropout': 0.3267140666269757, 'lr': 0.00025937129423859296}

Therefore, I will use the hypoparameter of 'hidden_size': 185, 'dropout': 0.3267140666269757, 'lr': 0.00025937129423859296 for optimized training -- both for 30 d embedding and 768d bert embedding


## **Step 3 continued: Insights**

Did you find the hyperparameter search helpful? Does it help to increase the number of trials in the optimization? Note that so far we have used the simplest version of optuna which has many nice features. Can you discover more useful features by browsing the optuna website? (Hint: try pruning)

The hyperparameter search is very helpdul for it could mark the exact best hyperparameters from the search range listed-- it saves a lost of time because there are a lot of hyperparameters to consider, but optuna could solve them once and for all. 

It actually would be helpful if I increase the trials in the optimization-- more trial times could mean higher possibilities of finding a better solution. For example, ther was a test run that the best outcome was in the 99th trial. 


module = optunahub.load_module(package="samplers/auto_sampler")
The package "samplers" looks very interesting.
Autosampler is a package that is loaded from OptunaHub, Optuna is a extension of Optuna used for simplize and automate the sampling method used in hyperparameter optimization. While AutoSampler could be used for choosing the suitable sampling method automatically for optimization.




Pruning is another thing that is very interesting.
This is the code:

from optuna.pruners import MedianPruner

study = optuna.create_study(direction="minimize", pruner=MedianPruner())
study.optimize(objective, n_trials=100)

This is one function that could do "early stopping" to end the experiment in advance if the training proformance is not as expected. 
It could save computational cost and avoid overfitting as it stops the experiment when the loss is increasing in a high rate.


## **Step 4: Final Training**

Now that you have found a good hyperparameter setting the validation set is no longer needed. The last step is to combine the training and validation set into a combined training set and retrain the model under the best parameter setting found. Report your final loss on your test data.

final test with 30d embedding

Average Training Loss: 0.4365

Test Error: 

 Accuracy: 78.5%, Avg loss: 0.425930

 The accuracy did not change because the demension reduction would lose some information that could be used for classification.

Final training with 768d bert embedding:

Epoch 1
-------------------------------
loss: 0.964580  [    0/ 5124]
loss: 0.566610  [  400/ 5124]
loss: 0.894764  [  800/ 5124]
loss: 0.741076  [ 1200/ 5124]
loss: 0.510969  [ 1600/ 5124]
loss: 0.402552  [ 2000/ 5124]
loss: 0.684575  [ 2400/ 5124]
loss: 0.250913  [ 2800/ 5124]
loss: 0.698820  [ 3200/ 5124]
loss: 0.480867  [ 3600/ 5124]
loss: 0.509612  [ 4000/ 5124]
loss: 0.483319  [ 4400/ 5124]
loss: 0.742746  [ 4800/ 5124]
Average Training Loss: 0.5555

Test Error: 
 Accuracy: 79.9%, Avg loss: 0.456661 

Epoch 2
-------------------------------
loss: 0.509154  [    0/ 5124]
loss: 0.479380  [  400/ 5124]
loss: 0.626528  [  800/ 5124]
loss: 0.397814  [ 1200/ 5124]
...

Average Training Loss: 0.4304

Test Error:

 Accuracy: 84.6%, Avg loss: 0.318552
  
For 100 epochs training with the optimized hyperparameters being evaluated by Optuna, the outcome is : the average training loss has dropped to 0.4304 from 0.556, which is obvious. It is obvious that the accuracy and loos drop is sharper in BERT embeddings-- as it increase 768 demensions each tensor, it contains more information for the models to learn and fit.

For test error, the accuracy has increased from 79.9% to 84.6%. 

The test average loss has dropped to 0.318 from 0.457. This is an obvious improvement.

Due to the data now used is text embedding, there are still space for improvement: according to previous studies by Hirakawa, M. (2024), Null subjects and long-distance anaphors revisited: What the acquisition of Japanese vs. Chinese contributes to generative approaches to SLA. Japanese Native speakers act very differently in terms of pronouns than English native speakers when speaking English. Therefore, a possible solution in improving the accuracy would be: fine tuning the BERT to have more attention on the patterns of pronouns of both Japanese Natives and English Natives dataset. Therefore, capturing and focus on the differeces on pronoun patterns.

Example: 
English Natives: 
I bought some apples in the store today.

Japanese Natives:
Some apples bought in the store today.



## **Final Submission**
Upload your submission for Milestone 2 to Canvas. 
Happy Deep Learning! 🚀